# Evaluate Model on Abstract Visual Reasoning Task
We use `Gemini` via the Google GenAI API to evaluate the model on the abstract visual reasoning task. We create a batch files that contain chunks of the test set. The input is the same as given to the meta-learning model, with an additional prompt that instructs the model with the respective task. The output should be the predicted output grid.

### Evaluate Batch Files
The following code evaluates the error types in the batch files. The batch files are stored in the `batch_files` directory. The code reads the batch files, sends the statements to the Gemini model, and evaluates the results. The results are stored in the `gemini` directory.

### Parameters

In [ ]:
IMAGE_INPUT = True
ONLY_FEW_SHOTS = False

In [ ]:
SEED = 1860
MODEL = "gemini-2.0-flash-001"

FILE_NAME = f"systematicity_seed_{SEED}"
BACTHFOLDER = "image_batch_files" if IMAGE_INPUT else "batch_files"
DATA_DIR = f"gs://mlc_bucket/{BACTHFOLDER}/split_seed_{SEED}"

if ONLY_FEW_SHOTS:
    DATA_DIR += "_only_few_shots"
DATA_DIR

In [ ]:
from pathlib import Path

# Paths
CURR_FILE_PATH = Path.cwd().resolve()
IMG_SPEC = "with_images" if IMAGE_INPUT else "text_only"
FEW_SHOT_SPEC = "only_few_shots" if ONLY_FEW_SHOTS else "vanilla"
OUT_DIR = f"gs://mlc_bucket/output/{MODEL}/{IMG_SPEC}/{FEW_SHOT_SPEC}"
OUT_DIR

### Keys

In [ ]:
import os
from dotenv import load_dotenv
from google import genai

load_dotenv()

# Retrieve the project ID key from environment variable
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")

# Check if the project ID key is retrieved successfully
if not PROJECT_ID:
    raise ValueError("Google project ID key not found. Ensure the GOOGLE_CLOUD_PROJECT environment variable is set correctly.")

LOCATION = "us-central1"

# set up client
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Script
Scripts to evaluate the error types in the models' responses.

#### Create Batch File

In [ ]:
INPUT_DATA = f"{DATA_DIR}/batch_file_samples_0-2499.jsonl"
BUCKET_URI = "gs://mlc_bucket/output"

In [ ]:
from google.genai.types import CreateBatchJobConfig

gcs_batch_job = client.batches.create(
    model=MODEL,
    src=INPUT_DATA,
    config=CreateBatchJobConfig(dest=OUT_DIR),
)
gcs_batch_job.name

### Fetch Job

In [ ]:
num_jobs = 4
latest_jobs = []
for i, job in enumerate(client.batches.list()):
    if i+1 > num_jobs:
        break
    latest_jobs.append(job)
    print(job.name, job.create_time, job.state)